# 13wk-1: 강화학습 (1) - Bandit

## 1. Imports

In [1]:
import numpy as np
import collections

## 2. 주요 코드 등

## 3. 강화학습 Intro

`-` 강화학습(대충설명): 어떠한 “(게임)환경”이 있을때 거기서 “뭘 할지”를
학습하는 과업

<figure class="margin-caption">
<img
src="https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig1.png?raw=true"
alt="그림1: 셔튼(Sutton, Barto, et al. (1998))의 교재에서 발췌한 그림, 되게 유명한 그림이에요" />
<figcaption aria-hidden="true">그림1: 셔튼(<span class="citation"
data-cites="sutton1998reinforcement">Sutton, Barto, et al.
(1998)</span>)의 교재에서 발췌한 그림, 되게 유명한
그림이에요</figcaption>
</figure>

> 게임환경 : Environment - Agent의 액션에 대한 보상
>
> 액션의 대상 : Agent - Environment에 대한 액션을 취함

`-` 딥마인드: breakout $\to$ 알파고

-   <https://www.youtube.com/watch?v=TmPfTpjtdgg>

<figure class="margin-caption">
<img
src="https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig2.png?raw=true"
alt="그림2: 벽돌깨기" />
<figcaption aria-hidden="true">그림2: 벽돌깨기</figcaption>
</figure>

> State : 현재의 상태. 실시간 데이터? 비전?

`-` 강화학습에서 “강화”는 뭘 강화한다는것일까?

-   <https://k9connoisseur.com/blogs/news/positive-reinforcement-dog-training>

> Agent의 행동을 강화함

`-` 강화학습 미래? (이거 잘하면 먹고 살 수 있을까?)

> 트랜스포머가 나오면서 그냥 싸잡아서 다 잘하는 애가 나오게 되었음
>
> 그럼에도 간간히 쓰임, 독특한 포지션
>
> 계속 바뀜, 그냥 바뀜

## 4. Bandit 게임 설명

`-` 문제설명: 두 개의 버튼이 있다. `버튼0`을 누르면 1의 보상을,
`버튼1`을 누르면 10의 보상을 준다고 가정

-   Agent: 버튼0을 누르거나,버튼1을 누르는 존재
-   Env: Agent의 Action을 바탕으로 Reward를 주는 존재

> **주의: 이 문제 상황에서 state는 없음**

`-` 생성형AI로 위의 상황을 설명한것

<table style="width:100%;">
<colgroup>
<col style="width: 33%" />
<col style="width: 33%" />
<col style="width: 33%" />
</colgroup>
<tbody>
<tr>
<td style="text-align: left;"><div width="33.3%"
data-layout-align="left">
<figure class="margin-caption">
<img
src="https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig3-gpt.png?raw=true"
alt="(a) 챗지피티로 생성한그림" />
<figcaption aria-hidden="true">(a) 챗지피티로 생성한그림</figcaption>
</figure>
</div></td>
<td style="text-align: left;"><div width="33.3%"
data-layout-align="left">
<figure class="margin-caption">
<img
src="https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig3-gemini.png?raw=true"
alt="(b) 제미나이로 생성한 그림" />
<figcaption aria-hidden="true">(b) 제미나이로 생성한 그림</figcaption>
</figure>
</div></td>
<td style="text-align: left;"><div width="33.3%"
data-layout-align="left">
<figure class="margin-caption">
<img
src="https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig3-perplexity.png?raw=true"
alt="(c) 퍼플렉시티로 생성한 그림" />
<figcaption aria-hidden="true">(c) 퍼플렉시티로 생성한 그림</figcaption>
</figure>
</div></td>
</tr>
</tbody>
</table>

-   클로드로 생성:
    <https://claude.ai/public/artifacts/1f52fcb2-ef08-4af1-8cf8-4a497d7bcc5f>

`-` 게임진행양상

-   처음에는 아는게 없음. 일단 “아무거나” 눌러보자. (“에이전트가
    랜덤액션을 한다” 고 표현함 )
-   한 20번 정도 눌러보면서 결과를 관찰함 (“에이전트가 경험을
    축적한다”고 표현함)
-   버튼0을 누를때는 1점, 버튼1을 누를때는 10점을 준다는 사실을 깨달음.
    (“에이전트가 환경을 이해했다”고 표현함)
-   버튼1을 누르는게 나한테 이득이 라는 사실을 깨달음. (“에이전트가
    최적의 정책을 학습했다” 고 표현함)
-   이제부터 무조건 버튼1만 누름 $\to$ 게임 클리어 (“강화학습 성공”이라
    표현할 수 있음)

`-` 어떻게 버튼1을 누르는게 이득이라는 사실을 아는거지? $\to$ 아래와
같은 테이블을 만들면 된다. (`q_table`)

|        |             Action0             |             Action1             |
|:------:|:-------------------------------:|:-------------------------------:|
| State0 | mean(Reward \| State0, Action0) | mean(Reward \| State0, Action1) |

> 조건부 기댓값으로 칠 때, 가장 추정치가 높은 액션을 선택하게 됨

## 5. Bandit 환경 설계 및 풀이

### **A. 대충 개념만 실습**

In [46]:
action_space = [0, 1] ## agent가 할 수 있는 action들의 집합
actions_queue = collections.deque(maxlen = 500) ## 액션들을 큐로 저장, 큐의 최대 길이는 500 -> 최근 500개 결과만 저장
rewards_queue = collections.deque(maxlen = 500) ## 리워드도 큐에 저장

* 처음엔 아는게 없어서 무작위 액션 - 에이전트가 랜덤액션을 함 + 에이전트가 경험을 축적함

In [47]:
for _ in range(10) :
    ## agent가 action을 선택
    action = np.random.choice(action_space)
    
    ## env가 reward를 제공
    if action == 0 :
        reward = 1
    elif action == 1 :
        reward = 10
    
    actions_queue.append(action)
    rewards_queue.append(reward)

In [49]:
## 큐를 행렬로 변환
actions_array = np.array(actions_queue)
rewards_array = np.array(rewards_queue)

In [51]:
q0 = rewards_array[actions_array == 0].mean()
q1 = rewards_array[actions_array == 1].mean()

q_table = np.array([q0, q1])
q_table

array([ 1., 10.])

In [53]:
action = q_table.argmax()
action

np.int64(1)

In [54]:
for _ in range(10) :
    action = q_table.argmax()

    if action == 0 :
        reward = 1
    else :
        reward = 10

    actions_queue.append(action)
    rewards_queue.append(reward)

    actions_array = np.array(actions_queue)
    rewards_array = np.array(rewards_queue)
    q0 = rewards_array[actions_array == 0].mean()
    q1 = rewards_array[actions_array == 1].mean()
    q_table = np.array([q0, q1])

In [56]:
actions_queue

deque([np.int64(0),
       np.int64(1),
       np.int64(0),
       np.int64(1),
       np.int64(1),
       np.int64(0),
       np.int64(0),
       np.int64(0),
       np.int64(0),
       np.int64(0),
       np.int64(1),
       np.int64(1),
       np.int64(1),
       np.int64(1),
       np.int64(1),
       np.int64(1),
       np.int64(1),
       np.int64(1),
       np.int64(1),
       np.int64(1)],
      maxlen=500)

### **B. 클래스를 이용한 설계 및 풀이**

In [60]:
class Bandit() :
    """
    Environment. 게임 그 자체
    """
    def __init__(self) :
        self.reward = None
        
    def step(self, action) :
        """
        function : action -> reward
        """
        if action == 0 :
            self.reward = 1
        else :
            self.reward = 10

        return self.reward

In [61]:
env = Bandit()

In [62]:
env.step(0)

1

In [63]:
env.reward

1

In [ ]:
class Agent :
    def __init__(self) :
        self.experience = 0

    def act(self) :
        if self.experience <= 20 :

    def save_experience(self) :

    def learn(self) :